In [1]:
import os
import time
import openpyxl
import locale
import pandas as pd
from loguru import logger
from datetime import datetime
from dotenv import load_dotenv
from sqlalchemy import create_engine, exc

load_dotenv()

True

In [2]:
class DatabaseConnection:
    def __init__(self):
        load_dotenv()
        self.DUSNEI_LOG_DIRECTORY = os.getenv('DUSNEI_LOG_DIRECTORY')
        self.LOG_FILE = os.path.join(self.DUSNEI_LOG_DIRECTORY, 'data.log')
        self.db_url = os.getenv('DUSNEI_URL')

    def initialize_logging(self):
        logger.add(
            sink= f'{self.DUSNEI_LOG_DIRECTORY}' + '/log_{time:YYYY-MM-DD}.log', 
            level='INFO', 
            rotation='1 day',
            format='{time:YYYY-MM-DD} | {function}: {message}'
        )

    def log_data(self):        
        for arquivo in os.listdir(self.DUSNEI_LOG_DIRECTORY):
            if arquivo.endswith('.log'):
                logger.info('Arquivo iniciado')

        
    def get_db_engine(self):
        try:
            self.db_url = os.getenv('DUSNEI_URL')
            engine = create_engine(self.db_url)
            # Test connection
            with engine.connect() as connection:
                logger.info('Conexão estabelecida!')
                pass
            logger.info('Banco de dados conectado!')
            return engine
        except exc.SQLAlchemyError as e:
            logger.info(f"Error: {e}")
            return None
        


In [ ]:
class PendenciasClientes():
    def __init__(self):
        load_dotenv()
        self.path_dados = os.getenv('DUSNEI_DATA_DIRECTORY_RELATORIO_PENDENCIAS')
        self.conn = DatabaseConnection.get_db_engine(self)
        self.supervisores = []
        
    def pendencias_clientes_query(self, conn, cod_supe):
        query = (f"""
            SELECT
                pfin.pfin_transacao AS transacao,
                pfin.pfin_status AS status,
                pfin.pfin_datavcto AS datavcto,
                pfin.pfin_pger_conta AS pger_conta,
                pfin.pfin_unid_codigo AS unid_cod,
                pfin.pfin_vend_codigo AS vend_cod,
                pfin.pfin_codentidade AS entidade_cod,
                pfin.pfin_cnpjcpf AS cnpj,
                pfin.pfin_numerodcto AS nota,
                pfin.pfin_parcela AS parcelas,
                pfin.pfin_valor AS valor,
                pfin.pfin_nparcelas AS nparcelas,
                pger.pger_descricao AS descricao_contabil,
                pger.pger_tipo AS tipo,
                clie.clie_nome AS clie_nome,
                clie.clie_razaosocial AS razaosocial,
                clie.clie_vend_codigo AS clie_vend_cod,
                clie.clie_foneres AS fone_res,
                clie.clie_fonecel AS fone_cel,
                vend.vend_nome AS vend_nome,
                vend.vend_supe_codigo AS supe_cod,
                CONCAT(vend.vend_extra16, vend.vend_extra15) AS vend_cel,
                unid.unid_reduzido AS unidade,
                supe.supe_nome AS supe_nome
            FROM pendfin AS pfin
            INNER JOIN planoger AS pger ON pfin.pfin_pger_conta = pger.pger_conta
            INNER JOIN unidades AS unid ON pfin.pfin_unid_codigo = unid.unid_codigo
            INNER JOIN clientes AS clie ON pfin.pfin_codentidade = clie.clie_codigo
            INNER JOIN vendedores AS vend ON clie.clie_vend_codigo = vend.vend_codigo
            INNER JOIN supervisores AS supe ON vend.vend_supe_codigo = supe.supe_codigo
            WHERE pfin.pfin_status = 'P'
            AND pfin.pfin_datavcto < CURRENT_DATE
            AND vend.vend_supe_codigo IN {tuple(cod_supe)}
            ORDER BY pfin.pfin_datavcto DESC
        """)
        df = pd.read_sql_query(query, conn)

        return df

    def save_to_excel(self, cod_supe):
        locale.setlocale(locale.LC_MONETARY, 'pt-BR.UTF-8')
        df = self.pendencias_clientes_query(self.conn, cod_supe)
        
        wb = openpyxl.Workbook()
        wb.remove(wb.active)

        # Agrupando o DataFrame por supervisor
        grouped_df = df.groupby('supe_cod')
        
        for supe_cod, group in grouped_df:
            ws = wb.create_sheet(title=f"Supervisor {supe_cod}")
        
            columns_title = [
                'Index','CodClie','RazaoSocial','CNPJ','Nota','Valor','Vencimento','Fone','Celular','CodSupe','Supervisor','CodVend','Vendedor','Celular','C.Descrição','Status','Hora'
            ]
            row_title = 1
        
            for col_num, column_title in enumerate(columns_title, 1):
                cell = ws.cell(row=row_title, column=col_num)
                cell.value = column_title

            for index, row in enumerate(group.itertuples(), start=2):
                status = "-"
                hora = "-"

                ws.cell(row=index, column=1).value = row.Index
                ws.cell(row=index, column=2).value = row.entidade_cod
                ws.cell(row=index, column=3).value = row.razaosocial
                ws.cell(row=index, column=4).value = row.cnpj
                ws.cell(row=index, column=5).value = row.nota
                ws.cell(row=index, column=6).value = row.valor
                ws.cell(row=index, column=7).value = row.datavcto.strftime("%Y/%m/%d")
                ws.cell(row=index, column=8).value = row.fone_res
                ws.cell(row=index, column=9).value = row.fone_cel
                ws.cell(row=index, column=10).value = row.supe_cod
                ws.cell(row=index, column=11).value = row.supe_nome
                ws.cell(row=index, column=12).value = row.clie_vend_cod
                ws.cell(row=index, column=13).value = row.vend_nome
                ws.cell(row=index, column=14).value = row.vend_cel
                ws.cell(row=index, column=15).value = row.descricao_contabil
                ws.cell(row=index, column=16).value = status
                ws.cell(row=index, column=17).value = hora

        data_pasta = datetime.now().strftime("%Y-%m-%d")
        nome_arquivo = (f'pendencias-{cod_supe}-{data_pasta}')
        diretorio = f'{self.path_dados}/{data_pasta}'
        if not os.path.exists(diretorio):
            os.mkdir(diretorio)
        local_arquivo = os.path.join(f'{diretorio}/{nome_arquivo}.xlsx')
        wb.save(local_arquivo)

  
if __name__ == "__main__":
    supervisor = ['001','003','004','007','008']
    teste = PendenciasClientes()
    teste.save_to_excel(supervisor)

In [ ]:
teste = PendenciasClientes()
cod_supe = ['001','003','004','007','008']

for codigo in cod_supe:
    connection = teste.conn
    df = teste.pendencias_clientes_query(connection, codigo)
    
    teste.save_to_excel(codigo)
    